<a href="https://colab.research.google.com/github/RitikesH-28/Week-5-Assignment/blob/main/Week_5_Assignment(Ritikesh_Bhardwaj).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pathway bokeh --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 4.6 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import pathway as pw
import bokeh.plotting as bk
from bokeh.models import HoverTool
import panel as pn
pn.extension()

# Step 1: Load and prepare the dataset
df = pd.read_csv('/content/dataset.csv')

# Combine date and time into a single datetime column
df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'],
                                  format='%d-%m-%Y %H:%M:%S')

# Sort by time and keep only necessary columns
df = df.sort_values('Timestamp').reset_index(drop=True)
df[['Timestamp', 'Occupancy', 'Capacity']].to_csv("parking_stream.csv", index=False)

# Step 2: Define Pathway schema
class ParkingSchema(pw.Schema):
    Timestamp: str
    Occupancy: int
    Capacity: int

# Load data using Pathway's replay_csv for streaming simulation
stream_data = pw.demo.replay_csv("parking_stream.csv", schema=ParkingSchema, input_rate=1000)

# Step 3: Add parsed datetime columns
fmt = "%Y-%m-%d %H:%M:%S"
stream_with_time = stream_data.with_columns(
    t=stream_data.Timestamp.dt.strptime(fmt),
    day=stream_data.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%dT00:00:00")
)

# Step 4: Apply daily tumbling window and calculate dynamic price
daily_window = (
    stream_with_time.windowby(
        pw.this.t,
        instance=pw.this.day,
        window=pw.temporal.tumbling(timedelta(days=1)),
        behavior=pw.temporal.exactly_once_behavior()
    )
    .reduce(
        t=pw.this._pw_window_end,
        occ_max=pw.reducers.max(pw.this.Occupancy),
        occ_min=pw.reducers.min(pw.this.Occupancy),
        cap=pw.reducers.max(pw.this.Capacity)
    )
    .with_columns(
        price=10 + (pw.this.occ_max - pw.this.occ_min) / pw.this.cap
    )
)

# Step 5: Define attractive plot with hover and styling
def price_plotter(source):
    fig = bk.figure(
        height=450,
        width=850,
        title="🚗 Dynamic Daily Parking Price Trend",
        x_axis_type="datetime",
        tools="pan,wheel_zoom,box_zoom,reset,save",
        background_fill_color="#f0f0f0",
        toolbar_location="above",
    )

    fig.line("t", "price", source=source, line_width=3, color="#0072B5", legend_label="Price")
    fig.scatter("t", "price", source=source, size=7, color="#D81B60", legend_label="Data Point")

    hover = HoverTool(
        tooltips=[
            ("Date", "@t{%F}"),
            ("Price", "@price{0.00}")
        ],
        formatters={"@t": "datetime"},
        mode="vline"
    )
    fig.add_tools(hover)

    fig.xaxis.axis_label = "Date"
    fig.yaxis.axis_label = "Dynamic Price (₹)"
    fig.title.text_font_size = "18pt"
    fig.title.text_color = "#333"
    fig.legend.location = "top_left"
    fig.grid.grid_line_alpha = 0.3

    return fig

# Step 6: Visualize using Pathway + Panel
viz = daily_window.plot(price_plotter, sorting_col="t")
pn.Column(viz).servable()


In [ ]:
# Start the Pathway pipeline execution in the background
# - This triggers the real-time data stream processing defined above
# - %%capture --no-display suppresses output in the notebook interface

%%capture --no-display
pw.run()